# Day 19

In [25]:
import numpy as np
from scipy.spatial.distance import pdist
# import matplotlib.pyplot as plt
import re
import plotly.express as px
import pandas as pd
# from IPython.display import display

In [26]:
example_input = """
--- scanner 0 ---
404,-588,-901
528,-643,409
-838,591,734
390,-675,-793
-537,-823,-458
-485,-357,347
-345,-311,381
-661,-816,-575
-876,649,763
-618,-824,-621
553,345,-567
474,580,667
-447,-329,318
-584,868,-557
544,-627,-890
564,392,-477
455,729,728
-892,524,684
-689,845,-530
423,-701,434
7,-33,-71
630,319,-379
443,580,662
-789,900,-551
459,-707,401

--- scanner 1 ---
686,422,578
605,423,415
515,917,-361
-336,658,858
95,138,22
-476,619,847
-340,-569,-846
567,-361,727
-460,603,-452
669,-402,600
729,430,532
-500,-761,534
-322,571,750
-466,-666,-811
-429,-592,574
-355,545,-477
703,-491,-529
-328,-685,520
413,935,-424
-391,539,-444
586,-435,557
-364,-763,-893
807,-499,-711
755,-354,-619
553,889,-390

--- scanner 2 ---
649,640,665
682,-795,504
-784,533,-524
-644,584,-595
-588,-843,648
-30,6,44
-674,560,763
500,723,-460
609,671,-379
-555,-800,653
-675,-892,-343
697,-426,-610
578,704,681
493,664,-388
-671,-858,530
-667,343,800
571,-461,-707
-138,-166,112
-889,563,-600
646,-828,498
640,759,510
-630,509,768
-681,-892,-333
673,-379,-804
-742,-814,-386
577,-820,562

--- scanner 3 ---
-589,542,597
605,-692,669
-500,565,-823
-660,373,557
-458,-679,-417
-488,449,543
-626,468,-788
338,-750,-386
528,-832,-391
562,-778,733
-938,-730,414
543,643,-506
-524,371,-870
407,773,750
-104,29,83
378,-903,-323
-778,-728,485
426,699,580
-438,-605,-362
-469,-447,-387
509,732,623
647,635,-688
-868,-804,481
614,-800,639
595,780,-596

--- scanner 4 ---
727,592,562
-293,-554,779
441,611,-461
-714,465,-776
-743,427,-804
-660,-479,-426
832,-632,460
927,-485,-438
408,393,-506
466,436,-512
110,16,151
-258,-428,682
-393,719,612
-211,-452,876
808,-476,-593
-575,615,604
-485,667,467
-680,325,-822
-627,-443,-432
872,-547,-609
833,512,582
807,604,487
839,-516,451
891,-625,532
-652,-548,-490
30,-46,-14
"""

In [27]:
beacons = []
scanner_name = 0
scanners = []

for line in example_input.split('\n'):
    if line == '':
        continue
    if '---' in line:
        scanner_name = int(re.findall('[0-9]+', line)[0])
        continue
    scanners.append(scanner_name)
    beacons.append([int(coord) for coord in line.split(',')])

beacons = np.array(beacons)
scanners = np.array(scanners)

num_beacon_distances = int(((beacons.shape[0]**2) - beacons.shape[0])/2)
print(f'Received {beacons.shape[0]} coordinates for beacons')
# print(f'You should expect a total of {num_beacon_distances} pairwise distances')

Received 127 coordinates for beacons


In [28]:
beacons_scanners = np.concatenate(
    (
        beacons, 
        scanners.reshape((beacons.shape[0],1)), 
        np.array([1] * beacons.shape[0]).reshape((beacons.shape[0], 1))
    ), 
    axis=1
)

beacons_scanners[:10]

array([[ 404, -588, -901,    0,    1],
       [ 528, -643,  409,    0,    1],
       [-838,  591,  734,    0,    1],
       [ 390, -675, -793,    0,    1],
       [-537, -823, -458,    0,    1],
       [-485, -357,  347,    0,    1],
       [-345, -311,  381,    0,    1],
       [-661, -816, -575,    0,    1],
       [-876,  649,  763,    0,    1],
       [-618, -824, -621,    0,    1]])

In [29]:
df = pd.DataFrame(beacons_scanners, columns=['x', 'y', 'z', 'scanner', 'size'])
df['scanner'] = df['scanner'].astype(str)
df.head()

,x,y,z,scanner,size
0,404,-588,-901,0,1
1,528,-643,409,0,1
2,-838,591,734,0,1
3,390,-675,-793,0,1
4,-537,-823,-458,0,1


## Old approach with pairwise distances

In [59]:
# scanners[np.where(scanners == 0)]

In [60]:
# beacon_distances = []

# for scanner in np.unique(scanners):
#     beacon_distances.append(pdist(beacons[np.where(scanners == scanner)], metric='euclidean'))

In [78]:
# xyz_names = df.loc[df['scanner'] == '0', 'x'].astype(str) + ',' + df.loc[df['scanner'] == '0', 'y'].astype(str) + ',' + df.loc[df['scanner'] == '0', 'z'].astype(str)

In [101]:
# pairwise_df = pd.DataFrame(np.zeros((25,25)))
# pairwise_df.columns = xyz_names
# pairwise_df.index = xyz_names
# pairwise_df

In [112]:
beacon_distances = []
for scanner in df['scanner'].unique():
    tmp_dist = pdist(df.loc[df['scanner'] == scanner, ['x', 'y', 'z']], metric='euclidean')
    beacon_distances.append(
        pd.DataFrame(    
            np.concatenate(
                (
                    tmp_dist.reshape((tmp_dist.shape[0], 1)),
                    np.array([int(scanner)] * tmp_dist.shape[0]).reshape((tmp_dist.shape[0], 1))
                ), 
                axis=1
            )
        )
    )

In [115]:
pairwise_df = pd.concat(beacon_distances, ignore_index=True)
pairwise_df.columns = ['Beacon Pairwise Distance', 'Scanner']
pairwise_df['Scanner'] = pairwise_df['Scanner'].astype(str)
pairwise_df.head()

,Beacon Pairwise Distance,Scanner
0,1317.004556,0.0
1,2367.663405,0.0
2,139.387948,0.0
3,1066.280920,0.0
4,1549.576071,0.0


In [117]:
pairwise_df.describe()

,Beacon Pairwise Distance
count,1550.000000
mean,1388.240338
std,518.052732
min,11.661904
25%,1122.397531
50%,1413.271903
75%,1762.834326
max,2525.269293


In [136]:
px.histogram(pairwise_df, color='Scanner', cumulative=False, labels={'value': 'Beacon Pairwise Distance'}, title='Pairs of Beacons Detected per Scanner')

In [135]:
pairwise_df.loc[pairwise_df['Beacon Pairwise Distance'] > 0, 'Beacon Pairwise Distance'].shape[0]#.nunique()

1550

`((N^2) - N)/2 = Y`

`N^2 - N - 2*Y = 0`

[calculate here](https://www.calculatorsoup.com/calculators/algebra/quadratic-formula-calculator.php)

- `N`: Number of beacons
- `Y`: Number of pairwise distances


In [155]:
inner_joins = []
skip_scanners = []

for left_scanner in pairwise_df['Scanner'].unique():
    left_df = pairwise_df.loc[pairwise_df['Scanner'] == left_scanner]
    for right_scanner in pairwise_df['Scanner'].unique():
        if (left_scanner == right_scanner) or (right_scanner in skip_scanners):
            continue
        right_df = pairwise_df.loc[pairwise_df['Scanner'] == right_scanner]
        inner_joins.append(
            left_df.merge(right_df, on='Beacon Pairwise Distance', how='inner', suffixes=('_left', '_right'))
        )
    skip_scanners.append(left_scanner)

In [156]:
pd.concat(inner_joins, ignore_index=True)[['Scanner_left', 'Scanner_right']].value_counts()

Scanner_left  Scanner_right
0.0           1.0              66
1.0           3.0              66
              4.0              66
2.0           4.0              66
0.0           4.0              15
1.0           2.0              15
3.0           4.0              15
0.0           2.0               3
2.0           3.0               3
dtype: int64

```
 4-3
 |\|
 0-1
```

In [80]:
all_beacon_distances = []

for dist in beacon_distances:
    all_beacon_distances.extend(dist)

unique_dist = np.unique(all_beacon_distances, return_counts=True)

In [81]:
(unique_dist[1] > 1).sum()

219

In [82]:
beacon_distances = pdist(beacons, metric='euclidean')
assert num_beacon_distances == beacon_distances.shape[0]
print(f'Calculated {beacon_distances.shape[0]} distances')

Calculated 8001 distances


In [99]:
np.mean(beacon_distances)

1371.0347170600016

In [100]:
px.histogram(beacon_distances, labels={'value': 'Pairwise Distance'}, title='Pairs of Beacons Detected', cumulative=True)

In [92]:
(np.unique(beacon_distances, return_counts=True)[0] <= 1000).sum()

1497

## Visualizin' the flips

In [93]:
# -x,  y,  z
#  x, -y,  z
#  x,  y, -z
# -x, -y,  z

test = [-1, 1, 1]

In [94]:
orientations = [f'{i:03b}' for i in range(8)]
print(orientations)

['000', '001', '010', '011', '100', '101', '110', '111']


In [95]:
coord_flips = np.array(
    [
        [int(coord.replace('1', '-1').replace('0', '1')) for coord in xyz] 
        for xyz in orientations
    ]
)
coord_flips

array([[ 1,  1,  1],
       [ 1,  1, -1],
       [ 1, -1,  1],
       [ 1, -1, -1],
       [-1,  1,  1],
       [-1,  1, -1],
       [-1, -1,  1],
       [-1, -1, -1]])

In [96]:
scanner_flips = [df.loc[df['scanner'] == '0']]

for flip in coord_flips:
    flipped = df.loc[df['scanner'] == '1', ['x', 'y', 'z']] * flip
    flipped = pd.concat(
        (
            flipped.reset_index(drop=True), 
            pd.Series([str(flip)] * df.loc[df['scanner'] == '1'].shape[0]),
            pd.Series([1] * df.loc[df['scanner'] == '1'].shape[0]),
        ),
        axis=1,
        ignore_index=True,
    )
    flipped.columns = ['x', 'y', 'z', 'scanner', 'size']
    scanner_flips.append(flipped)

In [97]:
flippin_df = pd.concat(scanner_flips, axis=0, ignore_index=True)
flippin_df.head()

,x,y,z,scanner,size
0,404,-588,-901,0,1
1,528,-643,409,0,1
2,-838,591,734,0,1
3,390,-675,-793,0,1
4,-537,-823,-458,0,1


In [98]:
px.scatter_3d(data_frame=flippin_df, x='x', y='y', z='z', color='scanner', size='size')